In [9]:
import torch
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from utils import load_embeddings
import wandb

In [10]:
wandb.init(entity="anitavero", project="finetune")
%env WANDB_SILENT=true

env: WANDB_SILENT=true


In [7]:
ids, embeddings, metadata = load_embeddings('/home/ubuntu/dev/detoxify/jigsaw-toxic-comment-classification-challenge/embeddings/embeddings_t5-large_test_This_text_is_about_{}.pkl')

In [11]:
labels = pd.read_csv('/home/ubuntu/dev/detoxify/jigsaw-toxic-comment-classification-challenge/data/test_labels.csv', dtype={"id": "string"})
classes = list(labels.columns)
classes.remove("id")
y = labels[classes].to_numpy()

In [13]:
mask = (y != -1).any(axis=1)
y_m = y[mask, :]
embeddings_m = embeddings[mask, :]
y_m.shape, embeddings_m.shape

X_train, X_test, y_train, y_test = train_test_split(embeddings_m, y_m, random_state=1)

In [14]:
clf = MLPClassifier(random_state=1, max_iter=300, verbose=True)
mo_clf = MultiOutputClassifier(estimator=clf).fit(X_train, y_train)

Iteration 1, loss = 0.22498170
Iteration 2, loss = 0.16746005
Iteration 3, loss = 0.15783564
Iteration 4, loss = 0.15290194
Iteration 5, loss = 0.14867438
Iteration 6, loss = 0.14538247
Iteration 7, loss = 0.14318401
Iteration 8, loss = 0.13958453
Iteration 9, loss = 0.13692006
Iteration 10, loss = 0.13452032
Iteration 11, loss = 0.13128942
Iteration 12, loss = 0.12774620
Iteration 13, loss = 0.12491148
Iteration 14, loss = 0.12226801
Iteration 15, loss = 0.11916301
Iteration 16, loss = 0.11563826
Iteration 17, loss = 0.11295411
Iteration 18, loss = 0.10987703
Iteration 19, loss = 0.10684017
Iteration 20, loss = 0.10471133
Iteration 21, loss = 0.10090460
Iteration 22, loss = 0.09811528
Iteration 23, loss = 0.09526682
Iteration 24, loss = 0.09283641
Iteration 25, loss = 0.08971750
Iteration 26, loss = 0.08722871
Iteration 27, loss = 0.08432123
Iteration 28, loss = 0.08162082
Iteration 29, loss = 0.07947609
Iteration 30, loss = 0.07703359
Iteration 31, loss = 0.07417719
Iteration 32, los

In [16]:
def predict_prob(embs):
    preds = mo_clf.predict_proba(embs)
    return np.column_stack([p[:, 1] for p in preds])

y_probs = predict_prob(X_test)
y_pred = mo_clf.predict(X_test)
score = mo_clf.score(X_test, y_test)

In [20]:
from model_eval.eval_predictions import evaluate

In [22]:
evaluate(y_probs, y_test)

{'auc_scores': [0.9313865491070665,
  0.9698520767649705,
  0.9377424028792546,
  0.9578011147486553,
  0.9373657230578611,
  0.9428209870325803],
 'mean_auc': 0.946161475598398}